In [1]:
# import required python modules
import pandas as pd
import os.path as os_path
import csv
import os
import re

In [2]:
# set global variables for start and end year -- these must be data type str
START_YEAR = '2015'
END_YEAR = '2022'

In [3]:
import numpy
print(numpy.__version__)
print(pd.__version__)
import sys
print(sys.version)

1.24.3
1.5.1
3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [4]:
# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory:", cwd)


Current working directory: C:\Users\user\Downloads


In [5]:
DATA_DIR = '..\data'
OUTPUT_DIR = '..\output'
PATEX_DIR = 'C:\\Users\\user\\Downloads\\BulkData2023'
PATENTSVIEW_DIR = "C:\\Users\\user\\Downloads\\data_patentsview_20231231"
OUTPUT_FNAME = 'nj_univ_appl_inv_gender.csv'

In [6]:
def id_rutgers_stevens_njit(df, text_col, output_col_suffix=''):
    '''Create columns to identify if text in 'text_col' is associated with Rutgers, Stevens, or NJIT

    This function is tailored for Rutgers University, Stevens Institute of Technology, and
    New Jersey Institute of Technology. Use a set of keywords (whole words or simplified) to
    identify if the text in 'text_col' contains the keywords assocated with each target
    university. A binary output column, one for each target university, identifies the result
    as True or False; to increase flexibility, add a suffix to the output columns.

    Rutgers = rutgers AND (univer OR jersey)
    Stevens = stevens AND institute
    NJIT = jersey AND institute AND technol

    :param df (dataframe): dataframe to analyze
    :param text_col (str): column containing text w/names
    :param output_col_suffix (str): suffix to add to end of output columns (default='')
    :return df (dataframe): dataframe with binary output columns indicating university
    
    # lowercase text in text_col
    df[text_col] = df[text_col].str.lower()
    # analyze Rutgers
    df['rutgers_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('rutgers' in x) and (('univer' in x) or ('jersey' in x)))
    # analyze Stevens
    df['stevens_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('stevens' in x) and ('institute' in x) and ('tech' in x))
    # analyze NJIT
    df['njit_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('jersey' in x) and ('institute' in x) and ('technol' in x))

    #analyse princeton
    df['princeton_'+output_col_suffix] = df[text_col].apply(lambda x: ('princeton' in x) and ('univer' in x))

    #analyse rowan
    df['rowan_'+output_col_suffix] = df[text_col].apply(lambda x: ('rowan' in x) and ('univer' in x))

    #analyse NJCU
    df['NJCU_'+output_col_suffix] = df[text_col].apply(lambda x: ('jersey' in x) and ('city' in x) and ('univer' in x))

    #analyse seton
    df['seton_'+output_col_suffix] = df[text_col].apply(lambda x: ('seton' in x) and ('hall' in x))

    #analyse saint peter's
    df['saintPeters_'+output_col_suffix] = df[text_col].apply(lambda x: ('saint') and ('peter' in x) and ('univer' in x))
    
    # return updated dataframe
    return df
    '''
   
    patterns = {
        'rutgers': r'\b(rutgers(?:,\s*the\s+state\s+university(?:\s+of\s+new\s+jersey)?)?)\b',
        'stevens': r'\b(stevens\s+insti(tute)?)\b',
        'njit': r'\b(jersey\s+insti(tute)?\s+of\s+tech(nology)?)\b',
        'princeton': r'\b(princeton\s+univer(sity)?)\b',
        'rowan': r'\b(rowan\s+univer(sity)?)\b',
        'njcu': r'\b(new\s+jersey\s+city\s+univer(sity)?)\b',
        'setonHall': r'\b(seton\s+hall\s+univer(sity)?)\b',
        'saintPeters': r'\b(sa(in)t\s*peter\s+univer(sity)?)\b'
    }

    # Apply each regex pattern to the text column and create a new binary column
    for university, pattern in patterns.items():
        column_name = university + '_' + output_col_suffix
        #df[column_name] = df[text_col].apply(lambda x: bool(re.search(pattern, str(x), re.IGNORECASE)))
        df[column_name] = df[text_col].apply(lambda x: bool(re.findall(pattern, str(x), re.IGNORECASE)))
        print(f'Created column: {column_name}')
        
    return df


In [7]:
def add_appl_owner_to_dict(appl_dict, appl_id, owner):
    '''Add the owner of an application to a dictionary.
    
    The dictionary key is an application number, and values are a set of owners for that application. 
    The use of sets precludes duplicates. Use data type to ignore blank values (e.g., NaN for an owner). 
    Note: we do not use a defaultdict to simplify the later conversion to a pandas dataframe.
    :param appl_dict (dict of sets): initial application owner dictionary 
    :param appl_id (str): application number
    :param owner (str): either an individual applicant or assignee of an application
    :return appl_dict (dict of sets): updated applicaton owner dictionary    
    '''
    # check if input application id is a string
    if type(appl_id)==str:
        # if application id is not in dictionary keys then add it to the dict w/empty set
        if appl_id not in appl_dict.keys(): appl_dict[appl_id] = set()
        # if input owner is a string, add to set for that application 
        if type(owner)==str: appl_dict[appl_id].add(owner)
    return appl_dict

In [8]:
# set filepath to PatEx applicant data csv file
filepath = os_path.join(PATEX_DIR, 'all_applicants.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\all_applicants.csv


In [9]:
with open(filepath, 'r') as file:
    for _ in range(2):
        print(file.readline(), end='')

application_number,applicant_organization,applicant_name_first,applicant_name_middle,applicant_name_last,applicant_city_name,applicant_region_code,applicant_country_code
10470614,,Societe Franco-Belge de Fabrication de,,Combustile - FBFC,Courbevoie,,FR


In [10]:
applicant_df = pd.read_csv(filepath,
                           header=0,
                           dtype=str,
                           keep_default_na=False)

In [11]:
applicant_df.loc[applicant_df['application_number'] == 17976614]

,application_number,applicant_organization,applicant_name_first,applicant_name_middle,applicant_name_last,applicant_city_name,applicant_region_code,applicant_country_code


In [12]:
# filter data to keep only US and NJ applicants
print('Number of rows before US NJ filter: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[(applicant_df['applicant_region_code']=='NJ')]

print('Number of rows after US NJ filter: {:,}'.format(len(applicant_df)))

Number of rows before US NJ filter: 6,752,288
Number of rows after US NJ filter: 155,377


In [13]:
# create column for application series code
applicant_df['series'] = applicant_df['application_number'].apply(lambda x: x[0:2])

In [14]:
# filter data to remove PCT applications -- begins with "PCT" and hence the series from above = "PC"
print('Number of rows before PCT filter: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[applicant_df['series']!='PC']

print('Number of rows after PCT filter: {:,}'.format(len(applicant_df)))

Number of rows before PCT filter: 155,377
Number of rows after PCT filter: 66,662


In [15]:
# create combined applicant string - each column separated by a space
# (note: using the python .join() function would be more complicated due to our using dataframes;
#        hence, use a simple concatenation)
applicant_df['applicant_combined'] = applicant_df['applicant_organization'] + ' ' + \
                                     applicant_df['applicant_name_first'] + ' ' + \
                                     applicant_df['applicant_name_middle'] + ' ' + \
                                     applicant_df['applicant_name_last']

In [16]:
applicant_df['count'] = applicant_df['applicant_organization'].str.split(',').apply(lambda x: len([item for item in x if 'INC' not in item]))

In [17]:
applicant_df[['applicant_combined']].to_excel('output_column.xlsx', index=False)

In [18]:
# identify applications having Rutgers, Stevens, or NJIT
applicant_df = id_rutgers_stevens_njit(applicant_df, 'applicant_combined', output_col_suffix='app')

Created column: rutgers_app
Created column: stevens_app
Created column: njit_app
Created column: princeton_app
Created column: rowan_app
Created column: njcu_app
Created column: setonHall_app
Created column: saintPeters_app


In [19]:
# summarize results
print('Number of Rutgers applications: {:,}'.format(len(applicant_df.loc[applicant_df['rutgers_app']==True])))
print('Number of Stevens applications: {:,}'.format(len(applicant_df.loc[applicant_df['stevens_app']==True])))
print('Number of NJIT applications: {:,}'.format(len(applicant_df.loc[applicant_df['njit_app']==True])))
print('Number of princeton applications: {:,}'.format(len(applicant_df.loc[applicant_df['princeton_app']==True])))
print('Number of rowan applications: {:,}'.format(len(applicant_df.loc[applicant_df['rowan_app']==True])))
print('Number of NJCU applications: {:,}'.format(len(applicant_df.loc[applicant_df['njcu_app']==True])))
print('Number of seton applications: {:,}'.format(len(applicant_df.loc[applicant_df['setonHall_app']==True])))
print('Number of saintPeters applications: {:,}'.format(len(applicant_df.loc[applicant_df['saintPeters_app']==True])))

Number of Rutgers applications: 1,136
Number of Stevens applications: 208
Number of NJIT applications: 230
Number of princeton applications: 741
Number of rowan applications: 124
Number of NJCU applications: 0
Number of seton applications: 14
Number of saintPeters applications: 0


In [20]:
# summarize the applicant names associated with the target universities
for univ in ['rutgers_app', 'stevens_app', 'njit_app','princeton_app','rowan_app','njcu_app','setonHall_app','saintPeters_app']:
    print('Applicant names for {}'.format(univ))
    print('----------------------')
    df = applicant_df.loc[applicant_df[univ]==True]
    print(df['applicant_combined'].value_counts())
    print()

Applicant names for rutgers_app
----------------------
Rutgers, The State University of New Jersey              822
RUTGERS, THE STATE UNIVERSITY OF NEW JERSEY              220
Rutgers, the State University of New Jersey               46
Rutgers University                                         7
Rutgers, The State University of New Jersey; Offic         5
Rutgers, The State University                              5
   Rutgers, The State University of New Jersey             4
Rutgers,The State University of New Jersey                 4
RUTGERS THE STATE UNIVERSITY OF NEW JERSEY                 3
Rutgers, the State University                              2
 RUTGERS, THE STATE UNIVERSITY OF  NEW JERSEY              2
RUTGERS UNIVERSITY                                         2
RUTGERS, THE STATE UNIVERSITY                              2
Rutgers, the State Univesity of New Jersey                 1
RUTGERS, the State University of New Jersey                1
Rutgers, The States University

In [21]:
## keep only observations w/target universities
print('Number of rows before removing non-target universities: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[(applicant_df['rutgers_app']==True) | \
                                (applicant_df['stevens_app']==True) | \
                                (applicant_df['njit_app']==True) | \
                                (applicant_df['princeton_app']==True) | \
                                (applicant_df['rowan_app']==True) | \
                                (applicant_df['njcu_app']==True) | \
                                (applicant_df['setonHall_app']==True) | \
                                (applicant_df['saintPeters_app']==True)]

print('Number of rows after removing non-target universities: {:,}'.format(len(applicant_df)))

Number of rows before removing non-target universities: 66,662
Number of rows after removing non-target universities: 2,453


In [22]:
# keep only the columns we need
applicant_df = applicant_df[['application_number','rutgers_app', 'stevens_app', \
                             'njit_app','princeton_app','rowan_app','njcu_app','setonHall_app','saintPeters_app']]

In [23]:
# create a consoldiated university variable
applicant_df.loc[applicant_df['rutgers_app']==True,'univ'] = 'Rutgers'
applicant_df.loc[applicant_df['stevens_app']==True,'univ'] = 'Stevens'
applicant_df.loc[applicant_df['njit_app']==True,'univ'] = 'NJIT'
applicant_df.loc[applicant_df['princeton_app']==True,'univ'] = 'Princeton'
applicant_df.loc[applicant_df['rowan_app']==True,'univ'] = 'Rowan'
applicant_df.loc[applicant_df['njcu_app']==True,'univ'] = 'NJCU'
applicant_df.loc[applicant_df['setonHall_app']==True,'univ'] = 'Seton'
applicant_df.loc[applicant_df['saintPeters_app']==True,'univ'] = 'saintPeters'
# print results
print(applicant_df['univ'].value_counts())

Rutgers      1136
Princeton     741
NJIT          230
Stevens       208
Rowan         124
Seton          14
Name: univ, dtype: int64


In [24]:
# define columns we want to use (see above)
'''
col_g_assignee  = ['patent_id','disambig_assignee_organization',
                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']
col_pg_assignee = ['pgpub_id','disambig_assignee_organization',
                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']
'''

"\ncol_g_assignee  = ['patent_id','disambig_assignee_organization',\n                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']\ncol_pg_assignee = ['pgpub_id','disambig_assignee_organization',\n                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']\n"

In [25]:
# set seperator for the data files; for tsv file, use '/t'
seperator = '\t'

In [26]:
# # open assignee data for patents and PGPubs as pandas dataframes
# # - identify the header as row 0 (the first row)
# # - specify columns we want to read via usecols option
# # - use seperator as specified above
# # - infer compression (default but specify this option anyway)
# # - ensure all columns are read in as strings
# # - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
# patent_assignee_df = pd.read_csv(os_path.join(PATENTSVIEW_DIR,'g_assignee_disambiguated.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_g_assignee, 
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)
# pgpub_assignee_df =  pd.read_csv(os_path.join(PATENTSVIEW_DIR,'pg_assignee_disambiguated.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_pg_assignee,
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)

In [27]:
# # as before, create a string combining all assignee name information
# patent_assignee_df['assignee_combined'] = patent_assignee_df['disambig_assignee_organization'] + ' ' + \
#                                           patent_assignee_df['disambig_assignee_individual_name_first'] + ' ' + \
#                                           patent_assignee_df['disambig_assignee_individual_name_last']
# pgpub_assignee_df['assignee_combined']  = pgpub_assignee_df['disambig_assignee_organization'] + ' ' + \
#                                           pgpub_assignee_df['disambig_assignee_individual_name_first'] + ' ' + \
#                                           pgpub_assignee_df['disambig_assignee_individual_name_last']

In [28]:
# # identify if target universities -- patents 
# patent_assignee_df = id_rutgers_stevens_njit(patent_assignee_df, 
#                                              'assignee_combined', 
#                                              output_col_suffix='asg_patent')

In [29]:
# # identify if target universities -- pgpubs
# pgpub_assignee_df  = id_rutgers_stevens_njit(pgpub_assignee_df, 
#                                              'assignee_combined', 
#                                              output_col_suffix='asg_pgpub')

In [30]:
# # summarize the assignee names associated with the target universities - patents
# for univ in ['rutgers_asg_patent', 'stevens_asg_patent', 'njit_asg_patent','princeton_asg_patent','rowan_asg_patent','seton_asg_patent','NJCU_asg_patent','saintPeters_asg_patent']:
#     print('Assignee names for {}'.format(univ))
#     print('----------------------')
#     df = patent_assignee_df.loc[patent_assignee_df[univ]==True]
#     print(df['disambig_assignee_organization'].value_counts())
#     print()

In [31]:
# # summarize the assignee names associated with the target universities - pgpubs
# for univ in ['rutgers_asg_pgpub', 'stevens_asg_pgpub', 'njit_asg_pgpub','princeton_asg_pgpub','rowan_asg_pgpub','NJCU_asg_pgpub','seton_asg_pgpub','saintPeters_asg_pgpub']:
#     print('Assignee names for {}'.format(univ))
#     print('----------------------')
#     df = pgpub_assignee_df.loc[pgpub_assignee_df[univ]==True]
#     print(df['disambig_assignee_organization'].value_counts())
#     print()

In [32]:
# # keep only observations w/target universities - patents
# print('PATENTS')
# print('Number of rows before removing non-target universities: {:,}'.format(len(patent_assignee_df)))

# patent_assignee_df = patent_assignee_df.loc[(patent_assignee_df['rutgers_asg_patent']==True) | \
#                                             (patent_assignee_df['stevens_asg_patent']==True) | \
#                                             (patent_assignee_df['njit_asg_patent']==True)    | \
#                                             (patent_assignee_df['princeton_asg_patent']==True) | \
#                                             (patent_assignee_df['rowan_asg_patent']==True) | \
#                                             (patent_assignee_df['NJCU_asg_patent']==True) | \
#                                             (patent_assignee_df['seton_asg_patent']==True) | \
#                                             (patent_assignee_df['saintPeters_asg_patent']==True)]
# print('Number of rows after removing non-target universities: {:,}'.format(len(patent_assignee_df)))

In [33]:
# # keep only observations w/target universities - pgpubs
# print('PGPUBS')
# print('Number of rows before removing non-target universities: {:,}'.format(len(pgpub_assignee_df)))

# pgpub_assignee_df = pgpub_assignee_df.loc[(pgpub_assignee_df['rutgers_asg_pgpub']==True) | \
#                                           (pgpub_assignee_df['stevens_asg_pgpub']==True) | \
#                                           (pgpub_assignee_df['njit_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['rowan_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['princeton_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['NJCU_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['seton_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['saintPeters_asg_pgpub']==True)]

# print('Number of rows after removing non-target universities: {:,}'.format(len(pgpub_assignee_df)))

In [34]:
# # keep only the columns we need 
# patent_assignee_df = patent_assignee_df[['patent_id','rutgers_asg_patent','stevens_asg_patent','njit_asg_patent','princeton_asg_patent','rowan_asg_patent','NJCU_asg_patent','seton_asg_patent','saintPeters_asg_patent']]
# pgpub_assignee_df = pgpub_assignee_df[['pgpub_id','rutgers_asg_pgpub','stevens_asg_pgpub','njit_asg_pgpub','rowan_asg_pgpub','princeton_asg_pgpub','NJCU_asg_pgpub','seton_asg_pgpub','saintPeters_asg_pgpub']]

In [35]:
# # create a consoldiated university variable - patents
# patent_assignee_df.loc[patent_assignee_df['rutgers_asg_patent']==True,'univ_asg_patent'] = 'Rutgers'
# patent_assignee_df.loc[patent_assignee_df['stevens_asg_patent']==True,'univ_asg_patent'] = 'Stevens'
# patent_assignee_df.loc[patent_assignee_df['njit_asg_patent']==True,'univ_asg_patent'] = 'NJIT'
# patent_assignee_df.loc[patent_assignee_df['princeton_asg_patent']==True,'univ_asg_patent'] = 'Princeton'
# patent_assignee_df.loc[patent_assignee_df['rowan_asg_patent']==True,'univ_asg_patent'] = 'Rowan'
# patent_assignee_df.loc[patent_assignee_df['NJCU_asg_patent']==True,'univ_asg_patent'] = 'NJCU'
# patent_assignee_df.loc[patent_assignee_df['seton_asg_patent']==True,'univ_asg_patent'] = 'Seton'
# patent_assignee_df.loc[patent_assignee_df['saintPeters_asg_patent']==True,'univ_asg_patent'] = 'SaintPeters'

# # print results 
# print('PATENTS')
# print(patent_assignee_df['univ_asg_patent'].value_counts())

In [36]:
# # create a consoldiated university variable - pgpubs
# pgpub_assignee_df.loc[pgpub_assignee_df['rutgers_asg_pgpub']==True,'univ_asg_pgpub'] = 'Rutgers'
# pgpub_assignee_df.loc[pgpub_assignee_df['stevens_asg_pgpub']==True,'univ_asg_pgpub'] = 'Stevens'
# pgpub_assignee_df.loc[pgpub_assignee_df['njit_asg_pgpub']==True,'univ_asg_pgpub'] = 'NJIT'
# pgpub_assignee_df.loc[pgpub_assignee_df['princeton_asg_pgpub']==True,'univ_asg_pgpub'] = 'Princeton'
# pgpub_assignee_df.loc[pgpub_assignee_df['rowan_asg_pgpub']==True,'univ_asg_pgpub'] = 'Rowan'
# pgpub_assignee_df.loc[pgpub_assignee_df['NJCU_asg_pgpub']==True,'univ_asg_pgpub'] = 'NJCU'
# pgpub_assignee_df.loc[pgpub_assignee_df['seton_asg_pgpub']==True,'univ_asg_pgpub'] = 'Seton'
# pgpub_assignee_df.loc[pgpub_assignee_df['saintPeters_asg_pgpub']==True,'univ_asg_pgpub'] = 'SaintPeters'

# # print results 
# print('PGPUBS')
# print(pgpub_assignee_df['univ_asg_pgpub'].value_counts())

In [37]:
# # define columns we want to use (see above)
# col_g_appln = ['patent_id','application_id']
# col_pg_appln = ['pgpub_id','application_id']

In [38]:
# set separator for .tsv file
seperator = '\t'

In [39]:
# # open data for patents and PGPubs as pandas dataframes
# # - identify the header as row 0 (the first row)
# # - specify columns we want to read via usecols option
# # - use same seperator as above
# # - infer compression (default but specify this option anyway)
# # - ensure all columns are read in as strings
# # - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
# patent_appln_df = pd.read_csv(os_path.join(PATENTSVIEW_DIR,'g_application.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_g_appln, 
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)
# pgpub_appln_df =  pd.read_csv(os_path.join(PATENTSVIEW_DIR,'pg_published_application.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_pg_appln,
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)

In [40]:
# # left join assignee dataframe (left table) to application dataframe (right table) -- patents 
# # use indicator=True to filter results 
# patent_assignee_df = patent_assignee_df.merge(patent_appln_df, 
#                                               how='left', 
#                                               on='patent_id', 
#                                               indicator=True)
# # print merge results 
# patent_assignee_df['_merge'].value_counts()

In [41]:
# # keep _merge=='both'
# patent_assignee_df = patent_assignee_df.loc[patent_assignee_df['_merge']=='both']
# # drop _merge column
# patent_assignee_df.drop(columns=['_merge'], inplace=True)

In [42]:
# # left join assignee dataframe (left table) to application dataframe (right table) -- pgpubs 
# # use indicator=True to filter results 
# pgpub_assignee_df = pgpub_assignee_df.merge(pgpub_appln_df, 
#                                             how='inner', 
#                                             on='pgpub_id', 
#                                             indicator=True)
# # print merge results 
# pgpub_assignee_df['_merge'].value_counts()

In [43]:
# # keep _merge=='both'
# pgpub_assignee_df = pgpub_assignee_df.loc[pgpub_assignee_df['_merge']=='both']
# # drop _merge column
# pgpub_assignee_df.drop(columns=['_merge'], inplace=True)

In [44]:
# # print size of assignee dataframes 
# print('Number patents having target universities as assignee: {:,}'.format(len(patent_assignee_df)))
# print('Number PGPubs having target universities as assignee: {:,}'.format(len(pgpub_assignee_df)))

In [45]:
# initialize the dictionary
appl_dict = {}

In [46]:
# process applicant data by looping through each row of the dataframe
# and calling the function add_appl_owner_to_dict 
for i in range(0,len(applicant_df)): 
    appl_dict = add_appl_owner_to_dict(appl_dict, 
                                       applicant_df.iloc[i]['application_number'], 
                                       applicant_df.iloc[i]['univ'])

In [47]:
# # similarly process patent assignee data; note different column names 
# for i in range(0,len(patent_assignee_df)): 
#     appl_dict = add_appl_owner_to_dict(appl_dict, 
#                                        patent_assignee_df.iloc[i]['application_id'], 
#                                        patent_assignee_df.iloc[i]['univ_asg_patent'])

In [48]:
# # similarly process pgpub assignee data; note different column names 
# for i in range(0,len(pgpub_assignee_df)): 
#     appl_dict = add_appl_owner_to_dict(appl_dict, 
#                                        pgpub_assignee_df.iloc[i]['application_id'], 
#                                        pgpub_assignee_df.iloc[i]['univ_asg_pgpub'])

In [49]:
# to faciliate creating a dataframe, create two lists, one for application IDs and another for universities, 
# where the set of university names is converted to a list
appl_list = []
univ_list = []
for key, value in appl_dict.items():
    appl_list.append(key)
    univ_list.append(list(value))
# verify length of the two list are the same 
n_appl_list = len(appl_list)
n_univ_list = len(univ_list)
print('Length of application ID list: {:,}'.format(n_appl_list))
print('Length of university list: {:,}'.format(n_univ_list))
if n_appl_list!=n_univ_list: 
    raise ValueError('# applications do not equal length of university list')

Length of application ID list: 2,423
Length of university list: 2,423


In [50]:
# create dataframe 
nj_univ_df = pd.DataFrame({'application_number': appl_list, 'univ_list': univ_list})

In [51]:
# create column for application series code
nj_univ_df['series'] = nj_univ_df['application_number'].apply(lambda x: x[0:2]) 

In [52]:
# filter data to remove PCT applications -- begins with "PCT" and hence the series from above = "PC"
print('Number of rows before PCT filter: {:,}'.format(len(nj_univ_df)))
nj_univ_df = nj_univ_df.loc[nj_univ_df['series']!='PC']
print('Number of rows after PCT filter: {:,}'.format(len(nj_univ_df)))

Number of rows before PCT filter: 2,423
Number of rows after PCT filter: 2,423


In [53]:
# print series -- value_counts (pandas 1.5) outputs a pandas series whose index are the series codes
nj_univ_df['series'].value_counts().sort_index()

12      1
13     93
14    400
15    385
16    411
17    408
18     32
61     42
62    478
63    173
Name: series, dtype: int64

In [54]:
# print universities -- these will include those with multiple university owners 
nj_univ_df['univ_list'].value_counts().sort_index()

[NJIT]                   223
[Princeton]              734
[Rowan]                  122
[Rutgers]               1109
[Rutgers, NJIT]            7
[Rutgers, Princeton]       4
[Rutgers, Rowan]           2
[Seton]                   14
[Stevens]                192
[Stevens, Princeton]       2
[Stevens, Rutgers]        14
Name: univ_list, dtype: int64

In [55]:
# to illustrate the next step, look at application '14777191' as an example
print('Before pandas explode()')
print(nj_univ_df.loc[nj_univ_df['application_number']=='14777191'].transpose())

Before pandas explode()
                         539
application_number  14777191
univ_list             [NJIT]
series                    14


In [56]:
# use pandas explode() to break apart multiple universities into seperate rows (keeping same application ID)
print('Number rows prior to breaking apart university lists: {:,}'.format(len(nj_univ_df)))
nj_univ_df = nj_univ_df.explode('univ_list')
print('Number rows after breaking apart university lists: {:,}'.format(len(nj_univ_df)))

Number rows prior to breaking apart university lists: 2,423
Number rows after breaking apart university lists: 2,452


In [57]:
print('After pandas explode()')
print(nj_univ_df.loc[nj_univ_df['application_number']=='14777191'].transpose())

After pandas explode()
                         539
application_number  14777191
univ_list               NJIT
series                    14


In [58]:
# examine result universities -- all should be singular 
nj_univ_df['univ_list'].value_counts().sort_index()

NJIT          230
Princeton     740
Rowan         124
Rutgers      1136
Seton          14
Stevens       208
Name: univ_list, dtype: int64

In [59]:
# rename univ_list column to univ
nj_univ_df.rename(columns={'univ_list': 'univ'}, inplace=True)

In [60]:
# # delete unneeded dataframes and data structures 
# del applicant_df, patent_assignee_df, pgpub_assignee_df
# del appl_dict, appl_list, univ_list

In [61]:
# look at first 5 observations (do not use transpose to preserve column orientation)
# note that university names are not in a list 
nj_univ_df.head(5)

,application_number,univ,series
0,17435954,Rutgers,17
1,17914647,Rutgers,17
2,18005188,Princeton,18
3,18245256,Rutgers,18
4,12539049,NJIT,12


In [62]:
# look at the first 6 observations (3 applications) having more than one university owner 
# (option keep=False tags all duplicate observation, not just the first or last)
# (note: this command does not change the underlying dataset - duplicates are not dropped)
nj_univ_df.loc[nj_univ_df.duplicated(subset=['application_number'], keep=False)].head(6)

,application_number,univ,series
223,14348197,Stevens,14
223,14348197,Rutgers,14
472,62170455,Stevens,62
472,62170455,Rutgers,62
487,14751086,Stevens,14
487,14751086,Rutgers,14


In [63]:
# set filepath to PatEx application data csv file
filepath = os_path.join(PATEX_DIR, 'application_data.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\application_data.csv


In [64]:
# identify columns we want to use
col_list = ['application_number','filing_date','application_invention_type',
            'earliest_pgpub_number','earliest_pgpub_date','patent_number','patent_issue_date',
            'invention_title','small_entity_indicator','aia_first_to_file']

# open csv file as a pandas dataframe
# - identify the header as row 0 (the first row)
# - use only selected columns
# - ensure all columns are read in as strings (important for the application number)
# - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
appl_df = pd.read_csv(filepath,
                      header=0,
                      usecols=col_list,
                      dtype=str,
                      keep_default_na=False)

In [65]:
appl_df.head()

,application_number,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file
0,04453098,,,,,,,,UNDISCOUNTED,
1,04544040,,,,,,,,UNDISCOUNTED,
2,04577552,,,,,,,,UNDISCOUNTED,
3,04589670,,,,,,,,UNDISCOUNTED,
4,04578318,,,,,,,,UNDISCOUNTED,


In [66]:
# create column for filing year
appl_df['filing_year'] = appl_df['filing_date'].apply(lambda x: x[0:4])

In [67]:
appl_df.head()

,application_number,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,04453098,,,,,,,,UNDISCOUNTED,,
1,04544040,,,,,,,,UNDISCOUNTED,,
2,04577552,,,,,,,,UNDISCOUNTED,,
3,04589670,,,,,,,,UNDISCOUNTED,,
4,04578318,,,,,,,,UNDISCOUNTED,,


In [68]:
# filter data to keep applications between start and end dates

print('Number of rows before date filter: {:,}'.format(len(appl_df)))

appl_df = appl_df.loc[(appl_df['filing_year']>=START_YEAR) & \
                      (appl_df['filing_year']<=END_YEAR)]

print('Number of rows after date filter: {:,}'.format(len(appl_df)))


Number of rows before date filter: 14,100,378
Number of rows after date filter: 4,246,035


In [69]:
# left join nj_univ_df (left table) to appl_df (right table); use indicator=True to filter results 
nj_univ_df = nj_univ_df.merge(appl_df, 
                                how='left', 
                                on='application_number', 
                                indicator=True)

In [70]:
nj_univ_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year,_merge
0,17435954,Rutgers,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,17914647,Rutgers,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,18005188,Princeton,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,18245256,Rutgers,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,12539049,NJIT,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [71]:
merge_results_series = nj_univ_df['_merge'].value_counts()
print(merge_results_series)

print('\nMerge results: ')
print('Number appl w/target NJ university applicants within date range: {:,}'.
      format(merge_results_series['both']))
print('Number appl w/target NJ university applicants outside date range (or no appl data): {:,}'.
      format(merge_results_series['left_only']))

both          2049
left_only      403
right_only       0
Name: _merge, dtype: int64

Merge results: 
Number appl w/target NJ university applicants within date range: 2,049
Number appl w/target NJ university applicants outside date range (or no appl data): 403


In [72]:
# keep _merge=='both'
nj_univ_df = nj_univ_df.loc[nj_univ_df['_merge']=='both']

In [73]:
nj_univ_df['univ'].value_counts()

Rutgers      948
Princeton    614
NJIT         193
Stevens      168
Rowan        118
Seton          8
Name: univ, dtype: int64

In [74]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year,_merge
391,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015,both
392,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015,both
393,14592489,Rutgers,14,2015-01-08,Utility,US20150202322A1,2015-07-23,9566349,2017-02-14,INTESTINAL PEPTIDE TARGETING LIGANDS,SMALL,false,2015,both
394,14594844,Rutgers,14,2015-01-12,Utility,US20150196657A1,2015-07-16,9943606,2018-04-17,Dendritic Polypeptide-Based Nanocarriers for t...,SMALL,true,2015,both
395,14594853,Rutgers,14,2015-01-12,Utility,US20160199155A1,2016-07-14,,,Orthodontic Brackets,SMALL,true,2015,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,18085724,NJIT,18,2022-12-21,Utility,US20230149855A1,2023-05-18,,,Hollow Fiber Membrane Module for Direct Contac...,SMALL,true,2022,both
2436,18013368,Rutgers,18,2022-12-28,Utility,,,,,IDENTIFICATION OF ESTROGEN RECEPTOR POSITIVE (...,SMALL,Other,2022,both
2437,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022,both
2438,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022,both


In [75]:
# drop _merge column
nj_univ_df.drop(columns=['_merge'], inplace=True)

In [76]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
391,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015
392,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015
393,14592489,Rutgers,14,2015-01-08,Utility,US20150202322A1,2015-07-23,9566349,2017-02-14,INTESTINAL PEPTIDE TARGETING LIGANDS,SMALL,false,2015
394,14594844,Rutgers,14,2015-01-12,Utility,US20150196657A1,2015-07-16,9943606,2018-04-17,Dendritic Polypeptide-Based Nanocarriers for t...,SMALL,true,2015
395,14594853,Rutgers,14,2015-01-12,Utility,US20160199155A1,2016-07-14,,,Orthodontic Brackets,SMALL,true,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,18085724,NJIT,18,2022-12-21,Utility,US20230149855A1,2023-05-18,,,Hollow Fiber Membrane Module for Direct Contac...,SMALL,true,2022
2436,18013368,Rutgers,18,2022-12-28,Utility,,,,,IDENTIFICATION OF ESTROGEN RECEPTOR POSITIVE (...,SMALL,Other,2022
2437,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022
2438,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022


In [77]:
# reset index; do not insert previous index into dataframe
nj_univ_df.reset_index(drop=True, inplace=True)

In [78]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015
1,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015
2,14592489,Rutgers,14,2015-01-08,Utility,US20150202322A1,2015-07-23,9566349,2017-02-14,INTESTINAL PEPTIDE TARGETING LIGANDS,SMALL,false,2015
3,14594844,Rutgers,14,2015-01-12,Utility,US20150196657A1,2015-07-16,9943606,2018-04-17,Dendritic Polypeptide-Based Nanocarriers for t...,SMALL,true,2015
4,14594853,Rutgers,14,2015-01-12,Utility,US20160199155A1,2016-07-14,,,Orthodontic Brackets,SMALL,true,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,18085724,NJIT,18,2022-12-21,Utility,US20230149855A1,2023-05-18,,,Hollow Fiber Membrane Module for Direct Contac...,SMALL,true,2022
2045,18013368,Rutgers,18,2022-12-28,Utility,,,,,IDENTIFICATION OF ESTROGEN RECEPTOR POSITIVE (...,SMALL,Other,2022
2046,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022
2047,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022


In [79]:
# delete application dataframe 
del appl_df 

In [80]:
# print first five rows of data (use transpose for readability)
nj_univ_df.head(5).transpose()

,0,1,2,3,4
application_number,14589845,62099988,14592489,14594844,14594853
univ,Princeton,Rutgers,Rutgers,Rutgers,Rutgers
series,14,62,14,14,14
filing_date,2015-01-05,2015-01-05,2015-01-08,2015-01-12,2015-01-12
application_invention_type,Utility,Provisional,Utility,Utility,Utility
earliest_pgpub_number,US20150191765A1,,US20150202322A1,US20150196657A1,US20160199155A1
earliest_pgpub_date,2015-07-09,,2015-07-23,2015-07-16,2016-07-14
patent_number,9752175,,9566349,9943606,
patent_issue_date,2017-09-05,,2017-02-14,2018-04-17,
invention_title,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,Sustainable Supply of Recipe Components for Ce...,INTESTINAL PEPTIDE TARGETING LIGANDS,Dendritic Polypeptide-Based Nanocarriers for t...,Orthodontic Brackets


In [81]:
nj_univ_df

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,SMALL,true,2015
1,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,SMALL,Other,2015
2,14592489,Rutgers,14,2015-01-08,Utility,US20150202322A1,2015-07-23,9566349,2017-02-14,INTESTINAL PEPTIDE TARGETING LIGANDS,SMALL,false,2015
3,14594844,Rutgers,14,2015-01-12,Utility,US20150196657A1,2015-07-16,9943606,2018-04-17,Dendritic Polypeptide-Based Nanocarriers for t...,SMALL,true,2015
4,14594853,Rutgers,14,2015-01-12,Utility,US20160199155A1,2016-07-14,,,Orthodontic Brackets,SMALL,true,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,18085724,NJIT,18,2022-12-21,Utility,US20230149855A1,2023-05-18,,,Hollow Fiber Membrane Module for Direct Contac...,SMALL,true,2022
2045,18013368,Rutgers,18,2022-12-28,Utility,,,,,IDENTIFICATION OF ESTROGEN RECEPTOR POSITIVE (...,SMALL,Other,2022
2046,18013302,Princeton,18,2022-12-28,Utility,,,,,COMPOUNDS HAVING ANTICANCER ACTIVITY,SMALL,true,2022
2047,18003753,Princeton,18,2022-12-29,Utility,,,,,Ketogenic Diet and Ketone Supplementation for ...,SMALL,Other,2022


In [82]:
nj_univ_df.to_csv('valid.csv')

In [83]:
# set filepath to PatEx inventor data csv file
filepath = os_path.join(PATEX_DIR, 'all_inventors.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\all_inventors.csv


In [84]:
# open csv file as a pandas dataframe
# - identify the header as row 0 (the first row)
# - use all columns (do not specify usecols)
# - ensure all columns are read in as strings (important for the application number)
# - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
inv_df = pd.read_csv(filepath,
                     header=0,
                     dtype=str,
                     keep_default_na=False)

In [85]:
inv_df.columns

Index(['application_number', 'inventor_name_first', 'inventor_name_middle',
       'inventor_name_last', 'inventor_rank', 'inventor_city_name',
       'inventor_region_code', 'inventor_country_code'],
      dtype='object')

In [86]:
inv_df.head()

,application_number,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code
0,04840815,WILLIAM,D.,SCHAEFFER,1,POMONA,CA,US
1,05434252,HERMAN,,LEVIN,1,GLENVIEW,IL,US
2,05434252,ERIC,K.,MAXON,2,EVANSTON,IL,US
3,05603052,AGIS,F.,KYDONIEUS,1,NEW YORK,NY,US
4,05823902,WILLIAM,G.,HECKENHAUER,1,BUCYRUS,OH,US


In [87]:
# left join applicant_df (left table) to inv_df (right table); use indicator=True to check results 
nj_univ_appl_inv_df = nj_univ_df.merge(inv_df, 
                                       how='left', 
                                       on='application_number', 
                                       indicator=True)

In [88]:
nj_univ_appl_inv_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,...,aia_first_to_file,filing_year,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code,_merge
0,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,...,true,2015,Howard,A.,Stone,1,Princeton,NJ,US,both
1,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,...,true,2015,Knut,,Drescher,2,Princeton,NJ,US,both
2,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,...,true,2015,Bonnie,L.,Bassler,3,Princeton,NJ,US,both
3,14589845,Princeton,14,2015-01-05,Utility,US20150191765A1,2015-07-09,9752175,2017-09-05,SYSTEMS AND METHODS TO DETECT BIOFILM STREAMER...,...,true,2015,Yi,,Shen,4,Zurich,,CH,both
4,62099988,Rutgers,62,2015-01-05,Provisional,,,,,Sustainable Supply of Recipe Components for Ce...,...,Other,2015,Richard,E.,Riman,1,Belle Mead,NJ,US,both


In [89]:
nj_univ_appl_inv_df.shape

(6711, 21)

In [90]:
merge_results_series = nj_univ_appl_inv_df['_merge'].value_counts()
print(merge_results_series)

print('\nMerge results: ')
print('Number application-applicant observations with inventor data: {:,}'.
      format(merge_results_series['both']))
print('Number application-applicant observations missing inventor data: {:,}'.
      format(merge_results_series['left_only']))

both          6702
left_only        9
right_only       0
Name: _merge, dtype: int64

Merge results: 
Number application-applicant observations with inventor data: 6,702
Number application-applicant observations missing inventor data: 9


In [91]:
# keep _merge=='both'
nj_univ_appl_inv_df = nj_univ_appl_inv_df.loc[nj_univ_appl_inv_df['_merge']=='both']

In [92]:
# drop _merge column 
nj_univ_appl_inv_df.drop(columns=['_merge'], inplace=True)

In [93]:
nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].count()

univ
NJIT          594
Princeton    2138
Rowan         313
Rutgers      3083
Seton          23
Stevens       551
Name: application_number, dtype: int64

In [94]:
nj_univ_appl_inv_df['inv_id'] = nj_univ_appl_inv_df.groupby(['inventor_name_last',
                                                             'inventor_name_first',
                                                             'inventor_name_middle']).ngroup()

In [95]:
type(nj_univ_appl_inv_df['series'][0])

str

In [96]:
filtered_nj_univ_appl_inv_df = nj_univ_appl_inv_df[nj_univ_appl_inv_df['series'].isin(['60', '61', '62', '63', '64'])]

In [97]:
nonprov_nj_univ_appl_inv_df = nj_univ_appl_inv_df[~nj_univ_appl_inv_df['series'].isin(['60', '61', '62', '63', '64'])]

In [98]:
#filtered_nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].value_count()
filtered_nj_univ_appl_inv_df[filtered_nj_univ_appl_inv_df['univ'] == 'Seton'].groupby(by=['univ'])['application_number'].value_counts()

univ   application_number
Seton  62637445              3
       62532118              2
Name: application_number, dtype: int64

In [99]:
filtered_nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].nunique()

univ
NJIT          78
Princeton    128
Rowan         47
Rutgers      310
Seton          2
Stevens       66
Name: application_number, dtype: int64

In [100]:
# count number of unque inventors by university
nonprov_nj_univ_appl_inv_df.groupby(by=['univ'])['application_number'].nunique()

univ
NJIT         115
Princeton    483
Rowan         71
Rutgers      632
Seton          6
Stevens      102
Name: application_number, dtype: int64

In [101]:
nonprov_nj_univ_appl_inv_df.to_csv('final_results.csv')

In [102]:
# import gender_it_functions
from gender_it import gender_it_functions as gf

In [103]:
#gf.read_wgnd(path='gender_it//')

In [ ]:
# get gender attributions for inventors; set threshold to 0.60
nj_univ_appl_inv_gender_df = gf.get_gender(nj_univ_appl_inv_df, 
                                           name_column = 'inventor_name_first', 
                                           country_column = 'inventor_country_code', 
                                           treshold = 0.6)

Step 1 - reading the name-country-gender dictionary
reading the dictionnary.
Step 2 - reading the name-language-gender dictionary
reading the dictionnary.
downloading the dictionnary.


In [ ]:
nj_univ_appl_inv_df.head()

In [ ]:
nj_univ_appl_inv_gender_df.head()

In [ ]:
nj_univ_appl_inv_gender_df.columns

In [ ]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","gender","filing_year"]).size()

In [ ]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","gender","filing_year"])['inv_id'].nunique()

In [ ]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","application_invention_type","filing_year","gender"])['inv_id'].nunique()

In [ ]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","application_invention_type","filing_year","gender"])['application_number'].nunique()

In [ ]:
nj_univ_appl_inv_gender_df.to_csv('first_results_with_asg.csv')

In [ ]:
print(appl_dict)

In [ ]:
pwd

In [ ]:
!git init

In [ ]:
!git remote add origin https://github.com/megh221322/USPTO_Data_Analysis

In [ ]:
!git status

In [ ]:
!git add replica.ipynb

In [ ]:
!git commit -m "commit successful"

In [ ]:
!git push origin master